参考资料：[1.14. 半监督学习¶](http://sklearn.apachecn.org/cn/0.19.0/modules/label_propagation.html#label-propagation)

In [ ]:
import tables as tb
import numpy as np
%pylab inline

In [ ]:
path = 'E:/xdata/X.h5'
h5 = tb.open_file(path)

cifar = h5.root.cifar10

In [ ]:
testX = np.reshape(cifar.testX, (-1, 32*32*3)).astype(np.float)
trainX = np.reshape(cifar.trainX, (-1, 32*32*3)).astype(np.float)
y_test = np.asanyarray(cifar.testY)
y_train = np.asanyarray(cifar.trainY)

In [ ]:
np.bincount(y_train)  # 类别均衡

In [ ]:
from sklearn.preprocessing import StandardScaler

# 标准化处理
sc = StandardScaler()  
sc.fit(trainX.astype(np.float))   # 计算数据中的每个特征的样本均值和标准差
X_train_std = sc.transform(trainX)   # 使用样本均值和标准差做标准化处理
X_test_std = sc.transform(testX)

In [ ]:
from sklearn.linear_model import Perceptron

# random_state 每次迭代后初始化重排训练数据集
ppn = Perceptron(tol=40, eta0=0.1, random_state=1)   # tol 迭代次数，eta0 学习率
ppn.fit(X_train_std, y_train)   # 训练模型

In [ ]:
y_pred = ppn.predict(X_test_std)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(
        np.arange(x1_min, x1_max, resolution),
        np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(
            x=X[y == cl, 0],
            y=X[y == cl, 1],
            alpha=0.8,
            c=colors[idx],
            marker=markers[idx],
            label=cl,
            edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(
            X_test[:, 0],
            X_test[:, 1],
            c='',
            edgecolor='black',
            alpha=1.0,
            linewidth=1,
            marker='o',
            s=100,
            label='test set')

In [ ]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

In [ ]:
X_combined_std.shape

In [ ]:
plot_decision_regions(X=X_combined_std, y=y_combined,
                      classifier=ppn, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
#plt.savefig('images/03_01.png', dpi=300)
plt.show()